In [3]:
import os, glob, re, io, random, gensim, smart_open, logging
import numpy as np

from PyPDF2 import PdfReader
from pdfminer.high_level import extract_text as fallback_text_extraction
from pdfminer.high_level import extract_text_to_fp
from pdfminer.layout import LAParams
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile
from nltk.tokenize import word_tokenize


In [4]:
pdfReaders = []
pdfFiles = []
docLabels = []

# taggedData = []
# wordSets = []

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
rootDir = "/Users/tillman/t-root/dev/projects/2022/pdf-correlator/gitignored"
txtExtractDir = "/Users/tillman/t-root/dev/projects/2022/pdf-correlator/gitignored/txt-extractions/"
zoteroDir = '/Users/tillman/t-root/zotero/storage'



In [5]:
# read files
def read_files():
    os.chdir(rootDir)
    for file in glob.glob("**/*.pdf"):
        try: 
            pdfFiles.append(file)
            pdfReaders.append(PdfReader(file))
        except:
            print("!exception found!")
#             return
    print("pdf files read")
            
read_files()

!exception found!
pdf files read


In [6]:
# def extract_to_txt():
#     os.chdir(txtExtractDir)
#     text = ""
#     counter = 0
#     for i in pdfReaders:
#         counter =+ 1
#         with open(rootDir + '/' + i, 'rb') as f:
            
#             extract_text_to_fp(f, outputString, output_type='text')

        
#     with open("corpus.txt", 'w', encoding="utf-8") as corpus:
#         outputString.seek(0)
#         shutil.copyfileobj(outputString, corpus)


# extract_to_txt()



In [7]:
# def extract_to_txt():
#     os.chdir(txtExtractDir)
#     text = ""
#     counter = 0
#     laparams = LAParams(char_margin=20.0)
#     outputString = StringIO()
#     for i in pdfReaders:
#         counter =+ 1
#         with open("corpus.txt", 'w', encoding="utf-8") as corpus:
            
#             # add doc title to array for reference / tagging
#             docLabels.append(i.metadata.title)
#             print(i.metadata.title)
#             text += fallback_text_extraction(rootDir + "/" + pdfFiles[counter], laparams)
#             corpus.write(text)   
            


# extract_to_txt()

In [8]:
# def extract_to_txt():
#     os.chdir(txtExtractDir)
#     text = ""
#     counter = 0
#     for i in pdfReaders:
#         counter =+ 1
#         with open("corpus.txt", 'w', encoding="utf-8") as corpus:
            
#             # add doc title to array for reference / tagging
#             docLabels.append(i.metadata.title)
#             print(i.metadata.title)
#             for j in range(len(i.pages)):
#                 try: 
#                     text += i.getPage(j).extract_text()
# #                     text += re.sub('.\n', '')
        
#                 except:
#                     # format txt file so that each line is one sentence (doc2vec requirement)
#                     print("err1: >> pdfminer attempt")
#                     print(rootDir + "/" + pdfFiles[counter])
#                     text += fallback_text_extraction(rootDir + "/" + pdfFiles[counter])

                    
#             corpus.write(text)   

# extract_to_txt()

In [9]:
# ======== OLD CODE SPLITING BY SENTENCE

def extract_to_txt():
    os.chdir(txtExtractDir)
    pat0 = ('(?<!Dr)(?<!Esq)\. +(?=[A-Z])')
    pat1 = ('\.+(?=[A-Z])')
    pat2 = ('\.+(?=[0-9])')
    pat3 = ('\. +(?=[0-9])')
    pat4 = ('(?=[for a of the and to in])')
    
    patterns = [pat0, pat1, pat2, pat3, pat4]
    counter = 0
    text = ""
    for i in pdfReaders:
        counter += 1
        print(counter)
        with open(str([i.metadata.title]) + ".txt", 'w', encoding="utf-8") as file:
            
            # add doc title to array for reference / tagging
            docLabels.append(i.metadata.title)
            print(i.metadata.title)
            try:
                for j in range(len(i.pages)):
                    # format txt file so that each line is one sentence (doc2vec requirement)
                    text += i.getPage(j).extract_text()
                    text = re.sub(patterns[0], '.\n', text)
                    text = re.sub(patterns[1], '.\n', text)
                    text = re.sub(patterns[2], '.\n', text)
                    text = re.sub(patterns[3], '.\n', text)
                    text = re.sub(patterns[4], '', text)
                  
                    
            except Exception as exc:
                    print(">>>>> exception")
                    # format txt file so that each line is one sentence (doc2vec requirement)
                    text += fallback_text_extraction(rootDir + "/" + pdfFiles[counter])
                    text = re.sub(patterns[0], '.\n', text)
                    text = re.sub(patterns[1], '.\n', text)
                    text = re.sub(patterns[2], '.\n', text)
                    text = re.sub(patterns[3], '.\n', text)
                    text = re.sub(patterns[4], '', text)
            file.write(text)   


extract_to_txt()




1
pone.0099019 1..8
2
None
3
User attention and behaviour in virtual reality art encounter
4
Microsoft Word - CHI2018_LucidDreaming_v5.docx
5
Microsoft Word - 48710116.DOC
6
frvir-2022-779148 1..5
7
Making Art Therapy Virtual: Integrating Virtual Reality Into Art Therapy With Adolescents
8
None
9
Making Art Therapy Virtual: Integrating Virtual Reality Into Art Therapy With Adolescents
10
ShareVR: Enabling Co-Located Experiences for Virtual Reality between HMD and Non-HMD Users
11
None
>>>>> exception
12
The effects of visual context and individual differences on perception and evaluation of modern art and graffiti art
13
IAFOR Journal of Cultural Studies – Volume 6 – Issue 1 
14
g5grap.lo
15
User attention and behaviour in virtual reality art encounter
16
ShareVR: Enabling Co-Located Experiences for Virtual Reality between HMD and Non-HMD Users


In [10]:
class CorpusGen(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self, tokens_only=False):
        for fname in os.listdir(self.dirname):
            with smart_open.open(fname, encoding="iso-8859-1") as f:
                for i, line in enumerate(f):
                    tokens = gensim.utils.simple_preprocess(line, min_len=3, max_len=15, deacc=True)
                    if tokens_only:
                        yield tokens
                    else:
                        yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

trainCorpus = list(CorpusGen('/Users/tillman/t-root/dev/projects/2022/pdf-correlator/gitignored/txt-extractions'))




In [11]:
# establish a model and build the vocab

model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=1, epochs=40)
model.build_vocab(trainCorpus)


2022-09-28 10:56:44,849 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dm/m,d50,n5,w5,s0.001,t3>', 'datetime': '2022-09-28T10:56:44.849206', 'gensim': '4.2.0', 'python': '3.10.6 | packaged by conda-forge | (main, Aug 22 2022, 20:41:22) [Clang 13.0.1 ]', 'platform': 'macOS-12.0.1-arm64-arm-64bit', 'event': 'created'}
2022-09-28 10:56:44,850 : INFO : collecting all words and their counts
2022-09-28 10:56:44,852 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2022-09-28 10:56:44,875 : INFO : PROGRESS: at example #10000, processed 60354 words (2751602 words/s), 9234 word types, 0 tags
2022-09-28 10:56:44,889 : INFO : PROGRESS: at example #20000, processed 130931 words (4927515 words/s), 10754 word types, 0 tags
2022-09-28 10:56:44,901 : INFO : PROGRESS: at example #30000, processed 179158 words (4286272 words/s), 10754 word types, 0 tags
2022-09-28 10:56:44,914 : INFO : PROGRESS: at example #40000, processed 245911 words (5293148 words/s), 11291 

In [39]:
# check word occurence
checkWord = "example"
print("\"" + str(checkWord) + "\"" + " appears this many times in corpus:")
print({model.wv.get_vecattr(checkWord, 'count')})

"example" appears this many times in corpus:
{782}


In [17]:
model.train(trainCorpus, total_examples=model.corpus_count, epochs=model.epochs)

2022-09-27 17:32:08,074 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 3 workers on 11292 vocabulary and 50 features, using sg=0 hs=0 sample=0.001 negative=5 window=5 shrink_windows=True', 'datetime': '2022-09-27T17:32:08.074471', 'gensim': '4.2.0', 'python': '3.10.6 | packaged by conda-forge | (main, Aug 22 2022, 20:41:22) [Clang 13.0.1 ]', 'platform': 'macOS-12.0.1-arm64-arm-64bit', 'event': 'train'}
2022-09-27 17:32:09,107 : INFO : EPOCH 0 - PROGRESS: at 53.81% examples, 331928 words/s, in_qsize 6, out_qsize 0
2022-09-27 17:32:09,904 : INFO : EPOCH 0: training on 624981 raw words (639474 effective words) took 1.8s, 352113 effective words/s
2022-09-27 17:32:10,911 : INFO : EPOCH 1 - PROGRESS: at 53.81% examples, 337519 words/s, in_qsize 5, out_qsize 0
2022-09-27 17:32:11,672 : INFO : EPOCH 1: training on 624981 raw words (639231 effective words) took 1.8s, 362547 effective words/s
2022-09-27 17:32:12,697 : INFO : EPOCH 2 - PROGRESS: at 56.06% examples, 342817 words/s, 

2022-09-27 17:33:04,754 : INFO : EPOCH 32 - PROGRESS: at 60.28% examples, 355532 words/s, in_qsize 6, out_qsize 0
2022-09-27 17:33:05,407 : INFO : EPOCH 32: training on 624981 raw words (639367 effective words) took 1.7s, 377056 effective words/s
2022-09-27 17:33:06,419 : INFO : EPOCH 33 - PROGRESS: at 58.72% examples, 357988 words/s, in_qsize 5, out_qsize 0
2022-09-27 17:33:07,100 : INFO : EPOCH 33: training on 624981 raw words (639440 effective words) took 1.7s, 378806 effective words/s
2022-09-27 17:33:08,126 : INFO : EPOCH 34 - PROGRESS: at 60.28% examples, 364267 words/s, in_qsize 5, out_qsize 0
2022-09-27 17:33:08,785 : INFO : EPOCH 34: training on 624981 raw words (639543 effective words) took 1.7s, 381194 effective words/s
2022-09-27 17:33:09,799 : INFO : EPOCH 35 - PROGRESS: at 58.72% examples, 357770 words/s, in_qsize 5, out_qsize 0
2022-09-27 17:33:10,478 : INFO : EPOCH 35: training on 624981 raw words (639384 effective words) took 1.7s, 379168 effective words/s
2022-09-27 1

In [18]:
# infer a vector from corupus (I dont actually know what this means or does! :D )
vector = model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])
print("infering a vector:")
print(vector)

infering a vector:
[ 0.29547322  0.65043265 -0.03726393  0.45231763  0.0445142  -0.14993511
 -0.116064    0.03431334 -0.19280624  0.06190386  0.2911727  -0.2891688
  0.25532177  0.13002183 -0.33655506  0.35234946 -0.35175878  0.36754876
 -0.9932178   0.15288362 -0.280485    0.47172755  0.3138533  -0.21402459
  0.04025637 -0.38199612 -0.5859955   0.24699849 -0.31489393  0.07576025
 -0.34178367  0.17578979 -0.15533762  0.01805525 -0.10024661  0.7873988
  0.20924246 -0.36599058 -0.42003304  0.1615636  -0.03278228 -0.14285877
  0.31852806 -0.18245164  0.26246023  0.279182    0.08503538 -0.01234088
 -0.21989544  0.15844044]


In [28]:
with open(txtExtractDir + "traincorpus.txt", 'w') as file:
    file.write(str(trainCorpus))

In [21]:
# assessing the model
ranks = []
secondRanks = []
for doc_id in range(len(model.dv)):
        inferredVector = model.infer_vector(trainCorpus[doc_id].words)
        sims = model.dv.most_similar([inferredVector], topn=len(model.dv))
        rank = [docid for docid, sim in sims].index(doc_id)
        ranks.append(rank)
        
        
        secondRanks.append(sims[1])

In [23]:
# for doc_id in range(len(trainCorpus)):
#     print(doc_id)

print(trainCorpus)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [24]:
import collections

counter = collections.Counter(ranks)
print(counter)


Counter({0: 3235, 1: 1142, 2: 393, 3: 245, 4: 170, 5: 145, 6: 124, 8: 91, 7: 85, 9: 76, 10: 68, 11: 58, 13: 44, 12: 42, 14: 35, 15: 30, 18: 29, 20: 27, 17: 24, 22: 23, 21: 18, 16: 18, 26: 18, 25: 17, 23: 17, 27: 16, 24: 16, 30: 16, 62: 16, 54: 15, 42: 15, 39: 15, 51: 14, 19: 13, 31: 13, 35: 13, 52: 12, 43: 11, 28: 11, 66: 11, 67: 11, 79: 11, 41: 11, 50: 11, 37: 10, 32: 10, 113: 10, 36: 10, 40: 10, 46: 9, 44: 9, 34: 9, 49: 9, 29: 9, 48: 9, 91: 9, 95: 9, 131: 8, 45: 8, 60: 8, 353: 8, 644: 8, 111: 8, 1089: 8, 521: 8, 440: 8, 586: 8, 470: 7, 1396: 7, 671: 7, 405: 7, 1103: 7, 114: 7, 73: 7, 651: 7, 117: 7, 92: 7, 2184: 7, 1162: 7, 798: 7, 522: 7, 193: 7, 58: 7, 47: 7, 118: 7, 33: 7, 219: 7, 554: 7, 356: 7, 312: 7, 683: 7, 156: 7, 146: 7, 68: 7, 1019: 7, 108: 7, 333: 7, 160: 7, 38: 7, 738: 7, 179: 7, 84: 6, 169: 6, 1615: 6, 102: 6, 1432: 6, 93: 6, 1098: 6, 795: 6, 546: 6, 257: 6, 215: 6, 524: 6, 789: 6, 399: 6, 277: 6, 106: 6, 1053: 6, 1076: 6, 216: 6, 70: 6, 1332: 6, 755: 6, 1150: 6, 832: 6

In [25]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(trainCorpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(trainCorpus[sims[index][0]].words)))

Document (17352): «»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec<dm/m,d50,n5,w5,s0.001,t3>:

MOST (13135, 0.5755646824836731): «»

SECOND-MOST (12236, 0.4984537661075592): «»

MEDIAN (5345, 0.00955571886152029): «participants had answer the question what extent you»

LEAST (17040, -0.47294923663139343): «narratology has»

